In [2]:
!pip install spacy_transformers
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00


In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import pandas as pd

In [4]:
spacy.__version__


'3.6.1'

In [5]:
!nvidia-smi


Thu Sep 14 08:57:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 7.67 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [7]:
cv_data=json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))

In [8]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

2023-09-14 08:57:31.670418: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
def get_spacy_doc(file,data):
  nlp=spacy.blank('en')
  db=DocBin()

  for text,annot in tqdm(data):
    doc=nlp.make_doc(text)
    annot=annot['entities']

    ents=[]
    entity_indices=[]

    for start,end,label in annot:
      skip_entity=False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue

      entity_indices=entity_indices+list(range(start,end))

      try:
        span = doc.char_span(start,end,label=label,alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data=str([start,end])+"  "+str(text)+'\n'
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass

  return db

In [10]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(cv_data,test_size=0.25)

In [11]:
len(train),len(test)

(150, 50)

In [12]:
file=open('error.txt','w')
db=get_spacy_doc(file,train)
db.to_disk('train_data.spacy')
db=get_spacy_doc(file,test)
db.to_disk('test_data.spacy')
file.close()

100%|██████████| 50/50 [00:00<00:00, 96.62it/s]


In [13]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train /content/train_data.spacy --paths.dev /content/test_data.spacy --gpu-id 0

2023-09-14 08:57:47.704361: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were

In [ ]:
nlp = spacy.load('/content/output/model-best')

In [15]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 55.9 MB/s eta 0:00:00


In [16]:
import sys,fitz

In [17]:
fname="/content/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.pdf"
doc=fitz.open(fname)

In [18]:
text=""
for page in doc:
  text=text+str(page.get_text())

In [19]:
' '.join(text.split())

'Alice Clark AI / Machine Learning Delhi, India Email me on Indeed • 20+ years of experience in data handling, design, and development • Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to data warehousing and business intelligence • Database: Experience in database designing, scalability, back-up and recovery, writing and optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL) Willing to relocate anywhere WORK EXPERIENCE Software Engineer Microsoft – Bangalore, Karnataka January 2000 to Present 1. Microsoft Rewards Live dashboards: Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping online. Microsoft Rewards members can earn points when searching with Bing, browsing with Microsoft Edg

In [20]:
doc=nlp(text)
for ent in doc.ents:
  print(ent.text,"---->",ent.label_)

Alice Clark ----> Name
Delhi ----> Location
Microsoft ----> Companies worked at
Software Engineer ----> Designation
Microsoft ----> Companies worked at
Microsoft ----> Companies worked at
Microsoft ----> Companies worked at
Microsoft ----> Companies worked at
Microsoft ----> Companies worked at
Microsoft ----> Companies worked at
Microsoft ----> Companies worked at
Indian Institute of Technology ----> College Name
2001 ----> Graduation Year
Machine Learning, Natural Language Processing, and Big Data Handling ----> Skills


In [21]:
doc=nlp("I am Aryan Sethi. I work at Microsoft. I have 3 years of working experience.")
for ent in doc.ents:
  print(ent.text,"---->",ent.label_)

Aryan Sethi ----> Name
Microsoft ----> Companies worked at


In [31]:
import pickle
pkl_filename="model.pkl"
with open(pkl_filename,'wb') as file:
  pickle.dump(nlp,file)

with open(pkl_filename,'rb') as file:
  pickle_model=pickle.load(file)

In [44]:
def nlpParser(text):
  pkl_filename="model.pkl"
  with open(pkl_filename,'rb') as f_in:
    model=pickle.load(f_in)

  doc=model(text)
  dict={}
  for ent in doc.ents:
    # print(ent.text,"---->",ent.label_)
    dict[ent.label_]=ent.text

  return dict

In [45]:
data=nlpParser(text)
data

{'Name': 'Alice Clark',
 'Location': 'Delhi',
 'Companies worked at': 'Microsoft',
 'Designation': 'Software Engineer',
 'College Name': 'Indian Institute of Technology',
 'Graduation Year': '2001',
 'Skills': 'Machine Learning, Natural Language Processing, and Big Data Handling'}